In [ ]:
# Copyright (c) 2020 Grid Dynamics International, Inc. All Rights Reserved
# http://www.griddynamics.com
# Classification level: PUBLIC
# Licensed under the Apache License, Version 2.0(the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# Id:          ML-DSL
# Project:     ML DSL
# Description: DSL to configure and execute ML/DS pipelines


<h1><center>ML DSL in Action</center></h1>

Google Cloud Platform: 
1. Google Dataproc
2. Google Cloud Storage
3. Google AI Platform

In [ ]:
import com.griddynamics.dsl.ml.mldsl as mldsl
from com.griddynamics.dsl.ml.settings.profiles import PySparkJobProfile, AIProfile, DeployAIProfile
from com.griddynamics.dsl.ml.settings.description import Platform

import importlib
import os
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

## Prepare Sequences from Original Text

### Text Sample

In [ ]:
!head -n 20 dev/aclImdb/test/pos/0_10.txt

### Tokenizer to Create Sequences 

### Execute Tokenizer on ML Platform

In [ ]:
%%py_script -e --name text_tokenizer.py --path demo/scripts --train_path dev/aclImdb/train/ --test_path dev/aclImdb/test/ --word_embeds dev/glove.6B.50d.txt -o demo
#!/usr/bin/python

from pyspark import SQLContext, SparkContext
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql.types import StringType, ArrayType, IntegerType, FloatType
from pyspark.ml.feature import Tokenizer
import argparse


def read_glove_vecs(glove_file, output_path):
    rdd = sc.textFile(glove_file)
    row = Row("glovevec")
    df = rdd.map(row).toDF()
    split_col = F.split(F.col('glovevec'), " ")
    df = df.withColumn('word', split_col.getItem(0))
    df = df.withColumn('splitted', split_col)
    vec_udf = F.udf(lambda row: [float(i) for i in row[1:]], ArrayType(FloatType()))
    df = df.withColumn('vec', vec_udf(F.col('splitted')))
    df = df.drop('splitted', "glovevec")
    w = Window.orderBy(["word"])
    qdf = df.withColumn('vec', F.concat_ws(',', 'vec')).withColumn("id", F.row_number().over(w))
    
    path = '{}/words'.format(output_path)
    qdf.coalesce(1).write.format('csv').option("sep","\t").option('header', 'true').save(path)
    print('Words saved to: "{}"'.format(path))
    list_words = list(map(lambda row: row.asDict(), qdf.collect()))
    word_to_vec_map = {item['word']: item['vec'] for item in list_words}
    words_to_index = {item['word']:item["id"] for item in list_words}
    index_to_words = {item["id"]: item['word'] for item in list_words}
    return words_to_index, index_to_words, word_to_vec_map


def prepare_df(path, const, words_dct):
    rdd = sc.textFile(path)
    row = Row("review")
    df = rdd.map(row).toDF()
    # Clean text
    df_clean = df.select(F.lower(F.regexp_replace(F.col('review'), "n't", " n't")).alias('review'))
    df_clean = df_clean.select(F.lower(F.regexp_replace(F.col('review'), 
                                                        "[^0-9a-zA-Z\\s]", "")).alias('review'))
    # Tokenize text
    tokenizer = Tokenizer(inputCol='review', outputCol='words_token')
    df_words_token = tokenizer.transform(df_clean).select('words_token')
    df_cutted = df_words_token.withColumn('length', F.size(F.col('words_token')))
    # Replace word with it's index
    word_udf = F.udf(lambda row: [words_to_index[w] if w in words_to_index.keys() else words_to_index["unk"] for w in row],
                 ArrayType(IntegerType()))
    df_stemmed = df_cutted.withColumn('words_stemmed', word_udf(F.col('words_token')))
    return df_stemmed.withColumn("class", F.lit(const))


def save_dataset(df_pos, df_neg, path):
    df = df_pos.union(df_neg)
    w = Window.orderBy(["words_stemmed"])
    df = df.withColumn("review_id", F.row_number().over(w)).withColumn('int_seq',
                                                                       F.concat_ws(',', 'words_stemmed'))
    qdf = df.select(['review_id', 'int_seq', 'class'])
    qdf.coalesce(1).write.format('csv').option('header', 'true').save(path)

    
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--train_path', type=str, help="train positive reviews path")
    parser.add_argument('--test_path', type=str, help="test positive reviews path")
    parser.add_argument('--word_embeds', type=str, help="Path to glove word embeddings")
    parser.add_argument('--output_path', type=str, help="Sequences output path")
    

    args, d = parser.parse_known_args()
    output_path = args.output_path
    SparkContext.setSystemProperty('spark.sql.broadcastTimeout', '36000')
    sc = SparkContext(appName="word_tokenizer").getOrCreate()
    sql = SQLContext(sc)

    words_to_index, index_to_words, word_to_vec_map = read_glove_vecs(args.word_embeds, output_path)
    reviews_filter = '999*.txt'
    df_pos = prepare_df(f"{args.train_path}/pos/{reviews_filter}", 1, words_to_index)
    df_neg = prepare_df(f"{args.train_path}/neg/{reviews_filter}", 0, words_to_index)
    train_path = '{}/train'.format(output_path)
    save_dataset(df_pos, df_neg, train_path)
    print('Train saved to: "{}"'.format(train_path))

    df_pos = prepare_df(f"{args.test_path}/pos/{reviews_filter}", 1, words_to_index)
    df_neg = prepare_df(f"{args.test_path}/neg/{reviews_filter}", 0, words_to_index)
    test_path = '{}/test'.format(output_path)
    save_dataset(df_pos, df_neg, test_path)
    print('Test saved to: "{}"'.format(test_path))

In [ ]:
platform = Platform.GCP
profile = PySparkJobProfile(root_path='demo/scripts',
                                  bucket='ai4ops',
                                  project='gd-gcp-techlead-experiments',
                                  cluster='ai4ops',
                                  region='global',
                                  ai_region='us-central1-a',
                                  job_prefix='demo_job',
                                  job_async=False)
profile.args=profile.load_profile_data("demo/spark_job_args_gcp.json")
PySparkJobProfile.set('JobProfile', profile)

In [ ]:
%py_data -n text_tokenizer.py -p JobProfile -pm $platform -o gs://ai4ops/mldsl/data

In [ ]:
#Use job_demo_job_1592575205 instance to browse job properties.
#job_demo_job_1592575205 = job_tracker['demo_job_1592575205']

# ML Model to Predict Positive or Negative Review

### Configure Additional Dependencies

### LSTM Train Script

In [ ]:
%%py_script -n mr_model.py -p demo/model/words/trainer -o dev/models --epochs 3 --train_path gs://ai4ops/mldsl/data/demo_job_1590767199/train --word_embeds gs://ai4ops/mldsl/data/demo_job_1590767199/words --seq_len 150
#!/usr/bin/python

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    import pandas as pd
    import numpy as np
    from tensorflow.python.lib.io import file_io
    from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
    from tensorflow.keras.models import Model
    from tensorflow.keras.callbacks import Callback
    from tensorflow.keras.optimizers import Adam
    import tensorflow as tf
    import os

    from uuid import uuid4
    import argparse
    import matplotlib
    if matplotlib.get_backend() in ['TkAgg', 'TkCairo']:
        matplotlib.use('agg')
    import matplotlib.pyplot as plt
    import seaborn as sns

class MetricCallback(Callback):
    def on_train_begin(self,logs={}):
        self.losses = []
        self.accuracies = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracies.append(logs.get('acc'))


def read_glove_vectors(glove_file):
    files = file_io.get_matching_files('{}/part*'.format(glove_file))
    for file in files:
        with file_io.FileIO(file, 'r') as f:
            word_to_vec_map = {}
            words_to_index = {}
        fl = f.readline()
        for line in f:
            line = line.strip().split('\t')
            word_to_vec_map[line[0]] = np.array(line[1].split(','), dtype=np.float64)
            words_to_index[line[0]] = int(line[2])
    return words_to_index, word_to_vec_map


def read_csv(path):
    files = file_io.get_matching_files('{}/part*'.format(path))
    pdf = []
    for file in files:
        with file_io.FileIO(file, 'r') as f:
            df = pd.read_csv(f)
            if df is not None and len(df) != 0:
                pdf.append(df)
    if len(pdf) == 0:
        return None
    return pd.concat(pdf, axis=0, ignore_index=True).reset_index()


def pretrained_embed_layer(word_to_vec_map, word_to_index, emb_dim):
    emb_matrix = np.zeros((len(word_to_index)+1, emb_dim))
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]
    
    return emb_matrix


def define_model(input_shape, emb_matrix, vocab_len, emb_dim, rnn_units, dropout=0.5):
    sentence_indices = Input(input_shape, dtype="int32")
    # Create the embedding layer pretrained with GloVe Vectors
    embedding_layer = Embedding(input_dim=vocab_len, trainable=False, output_dim=emb_dim)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    # Propagate sentence_indices through your embedding layer
    embeddings = embedding_layer(sentence_indices)
    X = LSTM(units=rnn_units, return_sequences=False)(embeddings)
    # Add dropout with a probability 
    X = Dropout(dropout)(X)
    # Propagate X through a Dense layer
    X = Dense(2)(X)
    # Add a softmax activation
    X = Activation("softmax")(X)
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    return model


def convert_to_one_hot(Y, C=2):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

def prepare_dataset(path, N, word_to_index):
    data = read_csv(path)
    data.dropna(inplace=True)
    data['int_seq'] = data['int_seq'].apply(lambda x: [int(i) for i in x.split(',')])
    l = data['int_seq'].apply(lambda x: len(x))
    print("Max sequence is set to {}".format(N))
    data['int_seq'] = data['int_seq'].apply(lambda x: (x + [word_to_index["unk"]] * N)[:N])
    ds_x = np.asarray(list(data["int_seq"]))
    ds_y = data["class"].values
    return ds_x, ds_y, l


def plot_metrics(callback, dir_to_save):
    f, axes = plt.subplots(1, 2, figsize=(18, 5))
    plt.style.use('seaborn')
    plt.rcParams['axes.titlesize'] = 16
    sns.lineplot(x=range(len(callback.losses)), y=callback.losses, ax=axes[0])
    axes[0].title.set_text("Loss")
    sns.lineplot(x=range(len(callback.accuracies)), y=callback.accuracies, ax=axes[1])
    axes[1].title.set_text("Accuracy")
    plt.tight_layout(.5)
    plt.savefig('{}'.format(dir_to_save))
    plt.show()


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--train_path', type=str, help="Train files path")
    parser.add_argument('--output_path', type=str, help="Models output path")
    parser.add_argument('--word_embeds', type=str, help="Models output path")
    parser.add_argument('--seq_len', type=int, help="Length of input sequence")
    parser.add_argument('--epochs', type=int, help="Number of epochs")
    args, d = parser.parse_known_args()

    word_to_index, word_to_vec_map = read_glove_vectors(args.word_embeds)
    N = args.seq_len
    train_x, train_y, l = prepare_dataset(args.train_path, N, word_to_index)
    train_y = convert_to_one_hot(train_y, C=2)
    NUM_EPOCS = args.epochs
    RNN_STATE_DIM = 32
    LEARNING_RATE = 0.01
    vocab_len = len(word_to_index) + 1
    emb_dim = word_to_vec_map["cucumber"].shape[0]
    emb_matrix = pretrained_embed_layer(word_to_vec_map, word_to_index, emb_dim)

    model = define_model((N, ), emb_matrix, vocab_len, emb_dim, RNN_STATE_DIM)
    print(model.summary())
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=LEARNING_RATE), metrics=['accuracy'])
    # fit model
    metrics = MetricCallback()
    a = model.fit(train_x, train_y, batch_size=1024, epochs=NUM_EPOCS, callbacks=[metrics], 
                  shuffle=True)

    tf.get_logger().setLevel('ERROR')
    # save the model to file
    local_dir = uuid4().hex
    file_io.recursive_create_dir(local_dir)
    local_path = f'{local_dir}/saved_model'
    tf.saved_model.save(model, local_path)
    local_path_chart = '{}/metrics.png'.format(local_dir)
    plot_metrics(metrics, local_path_chart)
    
    remote_dir = args.output_path
    remote_path = f'{remote_dir}/saved_model'
    remote_path_chart = f'{remote_dir}/metrics.png'
    if not remote_dir.startswith('gs://'):
        file_io.recursive_create_dir(remote_dir)
    file_io.copy(local_path_chart, remote_path_chart)
    tf.saved_model.save(model, remote_path)
    
    file_io.delete_recursively(local_dir)

In [ ]:
platform = Platform.GCP
profile = AIProfile(bucket='ai4ops', cluster='ai4ops', region='global', job_prefix='train',
                    root_path='demo/model/words', 
                    project='gd-gcp-techlead-experiments',
                    ai_region='us-central1', job_async=False,
                    package_name='trainer', package_dst='mldsl/packages',
                    scale_tier='BASIC', runtime_version='1.15', python_version='3.7')
profile.arguments = profile.load_profile_data("demo/train_args.json")
AIProfile.set('AIProfile', profile)

In [ ]:
%%py_script --name setup.py --path demo/model/words
# %py_load demo/model/words/setup.py
#!/usr/bin/python
from setuptools import setup, find_packages

REQUIRED_PACKAGES = ['Keras==2.0.4','matplotlib==2.2.4','seaborn==0.9.0']

setup(
    name='trainer',
    version='1.0',
    packages=find_packages(),
    install_requires=REQUIRED_PACKAGES,
    author='Grid Dynamics ML Engineer',
    author_email='griddynamics@griddynamics.com',
    url='https://griddynamics.com'
)


In [ ]:
%py_train -n mr_model.py -s demo/model/words -p AIProfile -pm $platform -o gs://ai4ops/mldsl

## Deployment of model

##### Parameters of deployment: name of model, version, artifacts (models saved as .joblib, .h5 etc), custom code

In [ ]:
platform = Platform.GCP
profile = DeployAIProfile(bucket='ai4ops', cluster='ai4ops', region='global', job_prefix='deploy',
                          root_path='demo/deploy', project='gd-gcp-techlead-experiments',
                          ai_region='us-central1', job_async=False,
                          runtime_version='1.15', python_version='3.7',
                          version_name='v1', is_new_model=True,
                          path_to_saved_model='gs://ai4ops/mldsl/train_1592767608/saved_model/')
profile.arguments = {
    "framework": "TENSORFLOW"
}
DeployAIProfile.set('AIProfile', profile)

In [ ]:
%py_deploy -n mldsl -p AIProfile -pm $platform

In [ ]:
#Use job_deploy_1593103768 instance to browse job properties.
#job_tracker['deploy_1593103768']

## Prediction

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    from utils import *

word_to_index, word_to_vec_map = read_glove_vectors("gs://ai4ops/mldsl/data/demo_job_1590767199/words")
df_x, df_y, l = prepare_dataset('gs://ai4ops/mldsl/data/demo_job_1590767199/test', 150, word_to_index)

In [ ]:
ints = np.random.choice(len(df_y), 100)

test_x = df_x[ints]
test_y = df_y[ints]
test = test_x.tolist()

In [ ]:
%py_test -n mldsl -p AIProfile -pm $platform -t $test

In [ ]:
activation=[[0.6321429014205933, 0.3678571879863739], [0.12713439762592316, 0.872865617275238], [0.024109704419970512, 0.9758903384208679], [0.003592864843085408, 0.9964070916175842], [0.9954433441162109, 0.004556715488433838], [0.9952955842018127, 0.004704390186816454], [0.005713205318897963, 0.994286835193634], [0.7208364009857178, 0.2791635990142822], [0.8376214504241943, 0.16237856447696686], [0.007366553880274296, 0.9926334023475647], [0.9722245335578918, 0.02777550183236599], [0.17860515415668488, 0.8213948011398315], [0.01433265209197998, 0.9856674075126648], [0.9497862458229065, 0.05021375045180321], [0.016234956681728363, 0.9837650656700134], [0.8964889049530029, 0.10351112484931946], [0.6874122023582458, 0.31258782744407654], [0.9943034648895264, 0.0056966147385537624], [0.9732365608215332, 0.02676350623369217], [0.1060587540268898, 0.8939412832260132], [0.9830858707427979, 0.01691410318017006], [0.8533444404602051, 0.14665552973747253], [0.008659862913191319, 0.9913401007652283], [0.15943501889705658, 0.8405649662017822], [0.4126940369606018, 0.5873059630393982], [0.005080858711153269, 0.9949191212654114], [0.9920275807380676, 0.007972449995577335], [0.3200126886367798, 0.6799872517585754], [0.9934780597686768, 0.0065219649113714695], [0.17479296028614044, 0.8252071142196655], [0.005040396470576525, 0.9949595928192139], [0.9938644766807556, 0.006135530304163694], [0.1501808762550354, 0.8498191237449646], [0.006657484453171492, 0.9933425784111023], [0.9937819838523865, 0.006217994261533022], [0.0026067732833325863, 0.9973932504653931], [0.7288512587547302, 0.2711487412452698], [0.9950405955314636, 0.004959406331181526], [0.9918374419212341, 0.008162522688508034], [0.03815029561519623, 0.9618497490882874], [0.9279369115829468, 0.07206311821937561], [0.4548940062522888, 0.5451059937477112], [0.9859738349914551, 0.014026147313416004], [0.717399001121521, 0.2826010584831238], [0.9061329960823059, 0.09386692196130753], [0.008032294921576977, 0.9919677376747131], [0.1301707625389099, 0.8698292374610901], [0.03836825117468834, 0.9616317749023438], [0.9717956185340881, 0.02820434421300888], [0.9852221608161926, 0.014777859672904015], [0.028501763939857483, 0.9714981913566589], [0.34971481561660767, 0.6502852439880371], [0.38276126980781555, 0.6172387003898621], [0.010509427636861801, 0.9894905686378479], [0.817579984664917, 0.18241997063159943], [0.042655665427446365, 0.9573442935943604], [0.04805301874876022, 0.9519469738006592], [0.454426109790802, 0.545573890209198], [0.05582763999700546, 0.9441723823547363], [0.723135769367218, 0.276864230632782], [0.09276805818080902, 0.9072319865226746], [0.004389461595565081, 0.9956105351448059], [0.9838841557502747, 0.016115805134177208], [0.005091487895697355, 0.9949085116386414], [0.8129076957702637, 0.1870923638343811], [0.9876847267150879, 0.01231524907052517], [0.9725201725959778, 0.02747986651957035], [0.15392236411571503, 0.8460776805877686], [0.49223628640174866, 0.5077638030052185], [0.08798418194055557, 0.9120157957077026], [0.17651809751987457, 0.8234819173812866], [0.05052492022514343, 0.949475109577179], [0.9047545194625854, 0.09524549543857574], [0.002965514548122883, 0.9970344305038452], [0.04132595658302307, 0.9586740732192993], [0.9963577389717102, 0.003642183030024171], [0.005217075813561678, 0.9947828650474548], [0.41338545083999634, 0.5866145491600037], [0.06867239624261856, 0.9313275814056396], [0.8458981513977051, 0.15410184860229492], [0.9945433139801025, 0.005456637125462294], [0.024829810485243797, 0.9751701951026917], [0.8619129061698914, 0.13808707892894745], [0.9948599338531494, 0.005140069406479597], [0.5833669304847717, 0.41663309931755066], [0.9939045310020447, 0.006095414981245995], [0.005308421794325113, 0.9946915507316589], [0.9916778802871704, 0.008322128094732761], [0.9943532943725586, 0.005646682344377041], [0.4925752282142639, 0.5074247717857361], [0.030627448111772537, 0.969372570514679], [0.9161972999572754, 0.08380267024040222], [0.996094286441803, 0.0039057156536728144], [0.9912528395652771, 0.008747115731239319], [0.0869859904050827, 0.9130139946937561], [0.9876487851142883, 0.012351155281066895], [0.9654543399810791, 0.0345456562936306], [0.17442718148231506, 0.8255727887153625], [0.9880678653717041, 0.01193215698003769], [0.9955512285232544, 0.004448742605745792]]

In [ ]:
preds=np.array(activation).argmax(axis=1)
conf_mat = confusion_matrix(test_y, preds)
fig, ax = plt.subplots(figsize=(8,6))
classes=['Negative', 'Positive']
sns.heatmap(conf_mat, annot=True, fmt='d',xticklabels=classes, yticklabels=classes, cmap=plt.cm.BuGn)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
target_names = ['negative review', 'positive review']
print(classification_report(test_y, preds, target_names=target_names))